In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamTrigger")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

In [ ]:
# pre-requistes

# we need a server running on port 9999, that provides text stream to spark
# spark stream is a client, that will connect ot a server running on port 9999

# terminal, 

# nc -lk 9999

In [ ]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client

In [ ]:
lineStreamDf.printSchema()
# show will not work for stream, it will cause error
#lineStreamDf.show() # good for batchs, not for stream

In [ ]:
# print data on console 
# for stream write, we should use writeStream
# for batch use write
# append is mode
# console is keyword, present console output
# writeStream is an action, which runs the whole show,
# reading data, trigger window etc by writeStream
# in general 1 sec trigger used if not mentioned, default

# trigger time 10 second processingTime
# 10 second micro batch interval
# echoOnconsole is query
echoOnconsole = lineStreamDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .trigger(processingTime='10 seconds')\
                .start()

# start basically start the streaming, listen for input
# as input arrives, it starts processing
# listen for jupyter console for the output

In [ ]:
# query id
echoOnconsole.id

In [ ]:
# auto generated or user given
echoOnconsole.name 

In [ ]:
# if any exception
echoOnconsole.exception

In [ ]:
echoOnconsole.recentProgress

In [13]:
# await for termination 
# block next operations until any termination
echoOnconsole.awaitTermination()
print("await after") # this statement won't be executed until stream close
# now you have to go to server, stop the server using Ctrl + C

KeyboardInterrupt: 

In [ ]:
print("next cell")